<center>
<h1>Fake news detection using graph neural network</h1>
<h2>SAGE GNN</h2>
<i><h6>Victor GOUBET | Nicholas Tagliapietra | Asma Bakhtiariazad</h6></i>
</center>


----

# Library

In [ ]:
import torch
import pickle as pk

from tqdm.notebook import tqdm
from torch.nn import Linear, LogSoftmax
from torch_geometric.loader import DataLoader
from tool_box import plot_hist, ext_UPFD, get_acc
from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool

# Import datasets

In [ ]:
datasets = pk.load(open('../data/datasets.pkl', 'rb'))

# Define the model

In [ ]:
class SAGE(nn.Module):
    def __init__(self, num_n_feature, num_g_feature, emb_size):
    
        super(SAGE, self).__init__()
        torch.manual_seed(42)
        
        self.in_layer = SAGEConv(num_n_feature, hidden_channels)
        self.conv1 = SAGEConv(num_n_feature, emb_size)
        self.conv2 = SAGEConv(emb_size, emb_size)
        self.conv3 = SAGEConv(emb_size, emb_size)
        self.conv4 = SAGEConv(emb_size, emb_size)
        
        i = 2
        if num_g_feature:
             self.lin_g = Linear(num_g_feature, emb_size)
             i = 3

        self.out_layer = Linear(i * emb_size, 2)
        self.act = LogSoftmax(dim=-1)
        

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x
    
    
    def full_forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x 

# Training

## Define Training

## Train